## Vision Transformer (ViT)

In this assignment we're going to work with Vision Transformer. We will start to build our own vit model and train it on an image classification task.
The purpose of this homework is for you to get familar with ViT and get prepared for the final project.

In [ ]:
import math

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# VIT Implementation

The vision transformer can be seperated into three parts, we will implement each part and combine them in the end.

For the implementation, feel free to experiment different kinds of setup, as long as you use attention as the main computation unit and the ViT can be train to perform the image classification task present later.
You can read about the ViT implement from other libary: https://github.com/huggingface/pytorch-image-models/blob/main/timm/models/vision_transformer.py and https://github.com/pytorch/vision/blob/main/torchvision/models/vision_transformer.py

## PatchEmbedding
PatchEmbedding is responsible for dividing the input image into non-overlapping patches and projecting them into a specified embedding dimension. It uses a 2D convolution layer with a kernel size and stride equal to the patch size. The output is a sequence of linear embeddings for each patch.

In [ ]:
class PatchEmbedding(nn.Module):
    def __init__(self, image_size, patch_size, in_channels, embed_dim):
        self.image_size = image_size
        self.patch_size = patch_size
        self.in_channels = in_channels
        self.embed_dim = embed_dim
        super().__init__()
        self.projection = nn.Conv2d(in_channels, embed_dim, kernel_size=patch_size, stride=patch_size) ## project to path

    def forward(self, x):
        x = self.projection(x)  ## B X C X H X W -> B X E X H X W
        return x

## MultiHeadSelfAttention

This class implements the multi-head self-attention mechanism, which is a key component of the transformer architecture. It consists of multiple attention heads that independently compute scaled dot-product attention on the input embeddings. This allows the model to capture different aspects of the input at different positions. The attention outputs are concatenated and linearly transformed back to the original embedding size.

In [ ]:
class MultiHeadSelfAttention(nn.Module):
    def __init__(self, embed_dim, num_heads):
        self.embed_dim = embed_dim
        self.head_dim = self.embed_dim // num_heads
        self.num_heads = num_heads
        assert embed_dim % num_heads == 0, 'dim should be divisible by num_heads'
        super().__init__()
        self.qkv = nn.Linear(self.embed_dim, embed_dim * 3)
        self.projection = nn.Linear(self.embed_dim, self.embed_dim)

    def forward(self, x):
        B, N, C = x.shape
        qkv = self.qkv(x)
        qkv = qkv.view(B, N, 3, self.num_heads, self.head_dim).permute(2, 0, 3, 1, 4)

        q, k, v = qkv.unbind(0)
        a = q @ k.transpose(-2, -1) * (1.0 / math.sqrt(self.head_dim))
        a = F.softmax(a, dim=-1)
        x = a @ v

        x = x.transpose(1, 2).reshape(B, N, C)

        # x = (a @ v).transpose(1, 2).reshape(B, N, C)
        # x = self.projection(x)
        return x










## TransformerBlock
This class represents a single transformer layer. It includes a multi-head self-attention sublayer followed by a position-wise feed-forward network (MLP). Each sublayer is surrounded by residual connections.
You may also want to use layer normalization or other type of normalization.

In [ ]:
class TransformerBlock(nn.Module):
    def __init__(self, embed_dim, num_heads, mlp_dim, dropout):
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.mlp_dim = mlp_dim
        super().__init__()
        self.dropout = dropout

        self.attention = MultiHeadSelfAttention(self.embed_dim, self.num_heads)
        self.norm_1 = nn.LayerNorm(self.embed_dim)
        self.norm_2 = nn.LayerNorm(self.embed_dim)
        self.relu   = torch.functional.F.relu
        self.dropout_1 = nn.Dropout(self.dropout)
        self.mlp = nn.Sequential(
            nn.Linear(self.embed_dim, self.mlp_dim),
            nn.Dropout(self.dropout),
            nn.GELU(),
            nn.Linear(self.mlp_dim, self.embed_dim),
            nn.Dropout(self.dropout)
        )


    def forward(self, x):
        res = x ## embed dim
        x = self.norm_1(x) ## embed dim
        x = self.attention(x)
        x = self.dropout_1(x)
        z = x + res
        z = self.norm_2(z)
        x = self.mlp(z)
        x = self.relu(x)
        return x + z




## VisionTransformer:
This is the main class that assembles the entire Vision Transformer architecture. It starts with the PatchEmbedding layer to create patch embeddings from the input image. A special class token is added to the sequence, and positional embeddings are added to both the patch and class tokens. The sequence of patch embeddings is then passed through multiple TransformerBlock layers. The final output is the logits for all classes

In [ ]:
class VisionTransformer(nn.Module):
    def __init__(self, image_size, patch_size, in_channels, embed_dim, num_heads, mlp_dim, num_layers, num_classes, dropout=0.1):
        self.image_size = image_size
        self.path_size = patch_size
        self.in_channels = in_channels
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.mlp_dim = mlp_dim
        self.num_layers = num_layers
        self.num_classes = num_classes
        self.dropout = dropout
        super().__init__()
        self.class_token = nn.Parameter(torch.zeros(1, 1, self.embed_dim)) ## added channel for class token of size (number of heads, 1, embedding dimension)
        N = (image_size // patch_size) ** 2 ## number of patches
        self.position_embedding = nn.Parameter(torch.zeros(1, N + 1, self.embed_dim)) ## positional embedding for class and patch token
        self.patch_embedding = PatchEmbedding(image_size, patch_size, in_channels, embed_dim)
        self.transformer_blocks = nn.ModuleList([
            TransformerBlock(self.embed_dim, num_heads, self.mlp_dim, dropout)
            for _ in range(num_layers)
        ])
        self.mlp_head = nn.Sequential(
            nn.LayerNorm(embed_dim),
            nn.Linear(embed_dim, num_classes)
        )
        self.final_layer_norm = nn.LayerNorm(embed_dim)
        self.mlp_head = nn.Linear(embed_dim, num_classes)
    def forward(self, x):

        B = x.shape[0] ## batch size
        x = self.patch_embedding(x) ## pass through patch embedding
        x = x.reshape(B, -1, self.embed_dim) ## reshape to (batch size, number of patches, embedding dimension)
        class_token = self.class_token.expand(B, -1, -1) ## expand class token to batch size
        x = torch.cat((class_token, x), dim=1) ## concatenate class token to the embedding
        x += self.position_embedding ## add positional embedding
        for transformer in self.transformer_blocks:
            x = transformer(x)
        z_0 = x[:, 0] ## take the class token
        z_0 = self.final_layer_norm(z_0) ## layer norm
        y_hat = self.mlp_head(z_0) ## mlp head
        return y_hat ## return that




## Let's train the ViT!

We will train the vit to do the image classification with cifar100. Free free to change the optimizer and or add other tricks to improve the training

In [ ]:
# Example usage:
image_size = 32
patch_size = 4
in_channels = 3
embed_dim = 128
num_heads = 4
mlp_dim = 256
num_layers = 4
num_classes = 100
dropout = 0.1

batch_size = 64

In [ ]:
model = VisionTransformer(image_size, patch_size, in_channels, embed_dim, num_heads, mlp_dim, num_layers, num_classes, dropout).to(device)
#input_tensor = torch.randn(1, in_channels, image_size, image_size).to(device)
# output = model(input_tensor)
import gc
torch.cuda.empty_cache()
gc.collect()


In [ ]:
# Load the CIFAR-100 dataset
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.Resize(image_size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = datasets.CIFAR100(root='./data', train=True, download=True, transform=transform_train)
testset = datasets.CIFAR100(root='./data', train=False, download=True, transform=transform_test)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

In [ ]:
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=0.001, betas = (0.9, 0.999) , weight_decay = 0.1 )
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

best_val_acc = 38.02

In [ ]:
best_val_acc

In [ ]:
# Train the model
num_epochs = 100
#scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=  num_epochs)
for epoch in range(num_epochs):
    model.train()
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        # print(labels.shape, labels)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()
        #print(loss.item())

        # TODO Feel free to modify the training loop youself.
    #scheduler.step()

    # Validate the model
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    val_acc = 100 * correct / total
    print(f"Epoch: {epoch + 1}, Validation Accuracy: {val_acc}%")

    # Save the best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "best_model.pth")

Please submit your best_model.pth with this notebook. And report the best test results you get.